In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
URL = "https://fantasy.premierleague.com/news/"
headers = {"User-Agent": "Mozilla/5.0 (compatible; FPL-Agent/1.0)"}

resp = requests.get(URL, headers=headers)
resp.raise_for_status()
html = resp.text
soup = BeautifulSoup(html, "lxml")


In [3]:
# See what kind of elements contain article links/titles
for a in soup.select("a[href*='/news/']")[:10]:
    print(a.get("href"), "|", a.get_text(strip=True))


https://www.premierleague.com/en/news/1820580/off-the-bench-can-you-ignore-cunha-in-fpl | FPL Podcast
https://www.premierleague.com/en/news/4345793/sign-up-now-to-mypremier-league/ | myPremierLeague


In [4]:
articles = []
seen = set()

for a in soup.select("a[href*='/news/']"):
    href = a.get("href", "")
    if not href or href in seen or "/news/" not in href:
        continue
    seen.add(href)
    title = a.get_text(" ", strip=True)
    if title:
        articles.append({
            "url": href if href.startswith("http") else f"https://fantasy.premierleague.com{href}",
            "title": title
        })

df_articles = pd.DataFrame(articles).drop_duplicates("url")
df_articles.head(10)


,url,title
0,https://www.premierleague.com/en/news/1820580/...,FPL Podcast
1,https://www.premierleague.com/en/news/4345793/...,myPremierLeague


In [6]:
df_articles

,url,title
0,https://www.premierleague.com/en/news/1820580/...,FPL Podcast
1,https://www.premierleague.com/en/news/4345793/...,myPremierLeague


In [8]:
first_url = df_articles.iloc[0]["url"]
print("Testing:", first_url)

r = requests.get(first_url, headers=headers)
r.raise_for_status()
soup_article = BeautifulSoup(r.text, "lxml")

# Try common article text containers
paras = soup_article.find_all("p")
text = "\n".join(p.get_text(strip=True) for p in paras)
print(text[:1000])  # print first 1000 characters


Testing: https://www.premierleague.com/en/news/1820580/off-the-bench-can-you-ignore-cunha-in-fpl
FPL experts Gianni Buttice and Prasun Singhal reflect on the key talking points from Gameweek 9
After a third successive win, is it now time to invest in Manchester United players inFantasy Premier League?
FPL experts Gianni Buttice and Prasun Singhal discuss the form of a number of their players such asBryan Mbeumo(£8.1m), who scored twice last weekend, andBruno Fernandes(£8.9m).
They also discuss if it is time to bring in AFC Bournemouth'sJunior Kroupi(£4.6m) following his third goal in the last two Gameweeks.
Running order:- Liverpool dilemma (02m 40s)- Magical Mateta (13m 58s)- Selling Gyokeres for Arsenal defence (18m 14s)- Man Utd options (24m 16s)- Cooling on Man City (31m 54s)- Keen on Kroupi (38m 08s)




In [9]:
records = []
for idx, row in df_articles.head(5).iterrows():
    url = row["url"]
    try:
        r = requests.get(url, headers=headers, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "lxml")
        text = " ".join(p.get_text(strip=True) for p in soup.find_all("p"))
        records.append({
            "title": row["title"],
            "url": url,
            "text_excerpt": text[:400]
        })
    except Exception as e:
        print("Error:", e)

pd.DataFrame(records)


,title,url,text_excerpt
0,FPL Podcast,https://www.premierleague.com/en/news/1820580/...,FPL experts Gianni Buttice and Prasun Singhal ...
1,myPremierLeague,https://www.premierleague.com/en/news/4345793/...,Join now for free to set your preferences and ...


In [10]:
df_articles.to_csv("../data/fpl_latest_articles.csv", index=False)
print("Saved to data/fpl_latest_articles.csv")


Saved to data/fpl_latest_articles.csv
